<a href="https://colab.research.google.com/github/osmarbraz/exemplos_gensim/blob/master/Exemplo_Gensim_en_Freebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo King e Queen(en) usando Gensim e Freebase em inglês(en)

Exemplo de embendding usando Gensim Python Library com execução através do Google Colaboratory.<br>
É Biblioteca open-source para modelagem de tópicos não supervisionados e processamento de linguagem.
Implementado em Python e Cython. 
Projetado para lidar com grandes volumes de texto, usando fluxo de dados e algoritmos on-line incrementais. 
Seu diferencial em relação a maioria dos outros software para aprendizado de máquina os quais visam apenas o processamento em memória.<br>
https://radimrehurek.com/gensim/





# Preparação do ambiente
Preparação do ambiente para executar o gensim.

##Tratamento de logs

In [ ]:
#Biblioteca de logging
import logging

#Formato da mensagem
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Parâmetros
Parâmetros da execução do teste

In [ ]:
#Id do arquivo no Google Drive
idArquivo = '0B7XkCwpI5KDYeFdmcVltWkhtbmM'

#Definição do nome do arquivo compactado do modelo
arquivoModeloCompactado = 'freebase-vectors-skipgram1000-en.bin.gz'

#Definição do nome do arquivo do modelo
arquivoModelo = 'freebase-vectors-skipgram1000-en.bin.gz'

logging.info('Parâmetros definidos!')

2020-03-16 20:10:34,069 : INFO : Parâmetros definidos!


##Instalação

Instalando o gensim no Google Colaboratory.

No Jupiter Notebook executar através "Anaconda Prompt".


In [ ]:
pip install -U gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.1)


Instando o PyDrive no Google Colaboratory.

No Jupiter Notebook executar através "Anaconda Prompt".

PyDrive é uma biblioteca de manipulação de arquivos para o google drive.<br>

https://colab.research.google.com/notebooks/io.ipynb#scrollTo=7taylj9wpsA2

In [ ]:
 pip install PyDrive

## Bibliotecas

Importando as bibliotecas de manipulação de dos arquivos pydrive.<br>
https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html#


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

#Biblioteca para autenticar o usuário
from google.colab import auth

#Biblioteca para acessar o sistema de arquivos
import os

logging.info('Import realizado!')

2020-03-16 20:10:39,767 : INFO : Import realizado!


## Autenticação do usuário

O usuário precisar conceder  permissão para o **Google Cloud SDK** acessar o seu Google Colaboratory.

Copie o código de autorização da URL apresentada na janela aberta e copie e cole na caixa do código de autorização.

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Download
Para usar o Gensim, você pode criar o seu vetor de palavras e treiná-lo ou utilizar um dataset previamente treinado.
Neste apresentação utilizamos o dataset fornecido  em https://github.com/3Top/word2vec-api.<br>
São dados do Google News com:<br>
*   300 dimensões
*   3M vocabulário


Link do arquivo no google drive:
https://drive.google.com/open?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM<br>

O arquivo do download será colocado na pasta "**/content**" do Google Colaboratory.<br> Opcionalmente pode ser feito o download para uma pasta no Google Drive. Neste casou é necessário montar uma pasta no Google Drive no Google Colaboratory para receber o arquivo.<br>

Este arquivo do modelo não precisa ser descompactado.

In [ ]:
#Verifica se o arquivo do modelo existe
if not os.path.exists(arquivoModeloCompactado):  
    #Realiza o download do arquivo
    downloaded = drive.CreateFile({'id':idArquivo})

    #Salva o arquivo no diretório 'content' do colab
    downloaded.GetContentFile(arquivoModeloCompactado)

    logging.info('Arquivo {0} baixado'.format(arquivoModeloCompactado))
else:
    logging.info('Arquivo {0} já existe'.format(arquivoModeloCompactado))

2020-03-16 20:11:09,835 : WARNING : file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFo

Mostra o diretório corrente e seus arquivos.

In [ ]:
!pwd
!ls -la

/content
total 2473456
drwxr-xr-x 1 root root       4096 Mar 16 20:11 .
drwxr-xr-x 1 root root       4096 Mar 16 20:07 ..
-rw-r--r-- 1 root root       2683 Mar 16 20:11 adc.json
drwxr-xr-x 1 root root       4096 Mar 16 20:11 .config
-rw-r--r-- 1 root root 2532785074 Mar 16 20:12 freebase-vectors-skipgram1000-en.bin.gz
drwxr-xr-x 1 root root       4096 Mar  3 18:11 sample_data


# Prepara o modelo
Carrega as bibliotecas e modelo pré treinado para executar os exemplos.

##Importando as bibliotecas do Gensim
Importa as biblitecas do Gensim para carregar o modelo pré treinado.

In [ ]:
from IPython.display import display, HTML
from IPython.display import Image
from jinja2 import Template

import warnings
warnings.filterwarnings('ignore')

import numpy as np  
import gensim

logging.info('Import realizado!')

2020-03-16 20:12:28,869 : INFO : Import realizado!


##Carregando o modelo

Carregando o modelo do arquivo compactado.

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format(arquivoModeloCompactado, binary=True)
logging.info('Modelo carregado!')

2020-03-16 20:12:28,874 : INFO : loading projection weights from freebase-vectors-skipgram1000-en.bin.gz
2020-03-16 20:14:14,403 : INFO : loaded (1422903, 1000) matrix from freebase-vectors-skipgram1000-en.bin.gz
2020-03-16 20:14:14,404 : INFO : Modelo carregado!


Mostra alguns dados do modelo carregado.

In [ ]:
print(model)
print('Word2Vec(vocab='+str(len(model.wv.vocab))+', size='+ str(model.wv.vectors.shape)+')')

Word2Vec(vocab=1422903, size=(1422903, 1000))


Exibindo os dados de uma palavra.

Necessário especificar '/en/' antes da palavra.

In [ ]:
model.most_similar('/en/king')

2020-03-16 20:14:14,419 : INFO : precomputing L2-norms of word weight vectors


[('/en/rubberband_man', 0.5420562624931335),
 ('/en/t_i_vs_t_i_p', 0.5418274402618408),
 ('/en/what_you_know', 0.5327495336532593),
 ('/en/the_adventures_of_b_o_b', 0.5324323177337646),
 ('/en/young_jeezy', 0.5242753028869629),
 ('/en/the_big_bang', 0.5147055387496948),
 ('/en/front_back', 0.5142571926116943),
 ('/en/the_inspiration_thug_motivation_102', 0.5115785598754883),
 ('/en/my_ghetto_report_card', 0.5100880861282349),
 ('/en/young_dro', 0.506653368473053)]

##Função de apresentação dos dados
Função para melhorar a apresentação dos dados dos exemplos.

In [ ]:
table_list_template = """ 
<table> 
        <tr>  
           <th>Palavra</th>   
           <th>Similaridade</th>   
        </tr>  
        {% for item in items%}  
        <TR>     
           <TD class="c1">{{item[0]}}</TD>     
           <TD class="c2">{{item[1]}}</TD>   
        </TR>   
        {% endfor %}
</table> 
"""
def most_similar(positive, negative):
    nn = model.most_similar(positive=positive, negative=negative)
    tmpl = Template(table_list_template)
    output_html = tmpl.render(items=nn)
    return HTML(output_html)

logging.info('Função criada!')

2020-03-16 20:14:23,603 : INFO : Função criada!


# Exemplos

**DICA**:<br>
Para palavras com sinais léxicos, utilizar o caractere "**u**" antes da mesma para codifica-la como utf-8.


## Comparando palavras diferentes

King + Woman – Man = ?

In [ ]:
most_similar(positive=['/en/king','/en/woman'], negative=['/en/man'])


Distância entre woman e man

In [ ]:
model.distance('/en/woman', '/en/man')

0.42907577753067017